# Bob

* Bob wants to receive a message from Alice.
* He can communicate with Alice over an unsecure, classical channel.
* They can also communicate over a quantum channel.

Bob establishes a classical channel to Alice. Anyone can listen in:

In [ ]:
# server = "broker.emqx.io"  # fallback server.
mqtt_broker_ip = "broker.emqx.io"

channel = "quantum_cryptography/classical_channel/#"

import paho.mqtt.client as mqtt

# The callback for when the client receives a CONNACK response from the server.
def on_connect(client, userdata, flags, rc):
    print("Connected with result code " + str(rc))

    # Subscribing in on_connect() means that if we lose the connection and
    # reconnect then subscriptions will be renewed.
    client.subscribe(channel)


# The callback for when a PUBLISH message is received from the server.
def on_message(client, userdata, msg):
    print("received:\t" + msg.topic + " " + str(msg.payload))


client = mqtt.Client()
client.on_connect = on_connect
client.on_message = on_message

client.connect(mqtt_broker_ip, 1883, 60)

**Step 1**: Bob listens for messages from Alice:

In [ ]:
client.loop_forever()

# setup

* wait for setup data from Alice

In [ ]:
import paho.mqtt.client as mqtt
import json

mqtt_broker_ip = "broker.emqx.io"
channel = "quantum_cryptography/classical_channel"

client = mqtt.Client()
client.connect(mqtt_broker_ip, 1883, 60)
setup_data = None


def on_message(client, userdata, msg):
    global setup_data
    try:
        setup_data = json.loads(msg.payload)
        client.disconnect()
    except:
        print("unable to parse json: ", msg.payload)


def on_connect(client, userdata, flags, rc):
    print("Connected with result code " + str(rc))
    client.subscribe(channel)


client.on_message = on_message
client.on_connect = on_connect

client.loop_forever()

print(setup_data["bits_to_exchange"])

# quantum chnannel

In [ ]:
from time import sleep
from qcrypt.hardware.filterwheel import Filterwheel
from qcrypt.hardware.lightsensor import Lightsensor

filterwheel = Filterwheel(username="Bob", mqtt_broker_ip="broker.emqx.io")

In [ ]:
filterwheel.rotate()
sleep(0.25)
filterwheel.rotate(clockwise=False)